In [1]:
# if i can add a resistance to movement then the acceleration can zero in on a point.

In [2]:
import pygame as pg
import random
import math
import numpy as np
import sys

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pg.init()

game_width = 800
game_height = 600
fps = 60

black = (0,0,0)
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)

max_vel = 5
max_acc = .05

screen = pg.display.set_mode((game_width,game_height))
clock = pg.time.Clock()

In [4]:
def get_draw_pos(x, size):
    pos = x-size/2
    return pos

def magnitude_calc(vector):
    x = 0
    for i in vector:
        x += i**2
    magnitude = x**0.5
    return(magnitude)
    
def normalise(vector):
    magnitude = magnitude_calc(vector)
    if magnitude != 0:
        vector = vector/magnitude
    return(vector)

In [5]:
class create_point():
    def __init__(self, pos_x, pos_y, size, color):
        self.position = np.array([pos_x,pos_y], dtype='float64')
        self.velocity = np.array([0,0], dtype='float64')
        self.acceleration = np.array([0,0], dtype='float64')
        self.max_vel = max_vel
        self.max_acc = max_acc
        self.color = color
        self.size = size
        self.age= 0
    
    def race_to(self, target):
        distance = np.add(target, -self.position)
        desired_vel = normalise(distance)*self.max_vel
        delta_vel = np.add(desired_vel, -self.velocity)
        self.acceleration = normalise(delta_vel)*self.max_acc
        
    def seek (self, target):
        dis_to_target = np.add(target, -self.position)
        desired_vel = normalise(dis_to_target)*self.max_vel
        delta_vel = np.add(desired_vel, -self.velocity)

        if np.array_equal(delta_vel, np.array([0,0], dtype='float64')):
            directional_acc = normalise(dis_to_target)*self.max_acc
        else:
            directional_acc = normalise(delta_vel)*self.max_acc

        time_to_stop = round(round(magnitude_calc(self.velocity),3)/round(magnitude_calc(directional_acc),3))

        time_to_stop = int(time_to_stop)
        dis_required_to_stop = np.array([0,0], dtype='float64')

        for i in range(time_to_stop-1):
            dis_required_to_stop = np.add(dis_required_to_stop, (i+1)*np.absolute(directional_acc))

        if magnitude_calc(dis_to_target) <= magnitude_calc(dis_required_to_stop):
            desired_vel = np.array([0,0], dtype='float64')
            delta_vel = np.add(desired_vel, -self.velocity)
            self.acceleration = normalise(delta_vel)*self.max_acc
        else:
            self.acceleration = directional_acc
            
    def stop_at(self, target): # not perfect. I dont know what happens to 
        dis_to_target = np.add(target, -self.position)
        dir_vel = normalise(dis_to_target)*self.max_vel
        delta_vel = np.add(dir_vel, -self.velocity)
        
        if magnitude_calc(dis_to_target) < 2*self.max_acc and magnitude_calc(self.velocity) < 2*self.max_acc:
            self.position = target
            self.velocity = np.array([0,0], dtype='float64')
            self.acceleration = np.array([0,0], dtype='float64')
            return
        
        if np.array_equal(delta_vel, np.array([0,0], dtype='float64')):
            dir_acc = normalise(dis_to_target)*self.max_acc
        else:
            dir_acc = normalise(delta_vel)*self.max_acc

        time_to_stop = round(round(magnitude_calc(self.velocity),3)/round(magnitude_calc(dir_acc),3))

        time_to_stop = int(time_to_stop)
        dis_required_to_stop = np.array([0,0], dtype='float64')

        for i in range(time_to_stop):
            dis_required_to_stop = np.add(dis_required_to_stop, (i+1)*np.absolute(dir_acc))

        if magnitude_calc(dis_to_target) <= magnitude_calc(dis_required_to_stop)+magnitude_calc(self.velocity):
            desired_vel = np.array([0,0], dtype='float64')
            delta_vel = np.add(desired_vel, -self.velocity)
            self.acceleration = normalise(delta_vel)*self.max_acc
        else:
            self.acceleration = dir_acc
    
    def print_attributes(self):
        print("Acc: "+str(self.acceleration), ", Vel: "+str(self.velocity), ", Pos: "+str(self.position))
        
    def update(self):
        if magnitude_calc(self.velocity) > self.max_vel:
            self.acceleration *= 0
            self.velocity = normalise(self.velocity)*self.max_vel
        else:
            self.velocity += self.acceleration
        self.position += self.velocity
        self.age += 1
    
    def draw(self):
        pg.draw.ellipse(screen, self.color, (get_draw_pos(self.position[0],self.size), get_draw_pos(self.position[1],self.size), self.size, self.size))
        

In [6]:
test = []

test.append(create_point(100,400,15,green))
test.append(create_point(150,300,15,red))

test[0].acceleration = np.array([-.1,-.1], dtype='float64')
test[0].velocity = np.array([-2,2], dtype='float64')

hit = False
hit2 = False
game_over = False
while not game_over:
# for i in range(50):
    screen.fill(black)

    target = test[1].position
    
#     print()
#     test[0].print_attributes()
    test[0].stop_at(target)
    
    for t in test:   
        t.update()
        t.draw()
    
    if np.array_equal(test[0].position, target) and hit and hit2:
        game_over = True
    if np.array_equal(test[0].position, target) and hit:
        hit2 = True
    if np.array_equal(test[0].position, target):
        hit = True
    
    pg.display.update()
    clock.tick(fps)
    
    for event in pg.event.get():

        if event.type == pg.QUIT:
            game_over == True
            print("EXITED")

            sys.exit()
print()
print(test[0].age)
pg.quit()
quit()


466
